In [1]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
# Cache model example 4

model = Network('model')

In [3]:
# Multi-level cache hierarchy
node = np.empty(4, dtype = object)
node[0] = Delay(model, 'CPU')
node[1] = Queue(model, 'L1Cache', SchedStrategy.FCFS)
node[2] = Queue(model, 'L2Cache', SchedStrategy.FCFS)
node[3] = Queue(model, 'Memory', SchedStrategy.FCFS)
jobclass = ClosedClass(model, 'Requests', 5, node[0], 0)

In [4]:
# Cache hit/miss service times
node[0].setService(jobclass, Exp(10.0))  # CPU processing
node[1].setService(jobclass, Exp(100.0))  # L1 cache access
node[2].setService(jobclass, Exp(50.0))   # L2 cache access
node[3].setService(jobclass, Exp(5.0))    # Memory access

In [5]:
# Cache hierarchy routing (hit/miss probabilities)
P = model.initRoutingMatrix()

# CPU to L1 cache
P.set(jobclass, jobclass, node[0], node[1], 1.0)

# L1 hit back to CPU, miss to L2
P.set(jobclass, jobclass, node[1], node[0], 0.8)  # L1 hit rate
P.set(jobclass, jobclass, node[1], node[2], 0.2)  # L1 miss rate

# L2 hit back to CPU, miss to memory
P.set(jobclass, jobclass, node[2], node[0], 0.9)  # L2 hit rate
P.set(jobclass, jobclass, node[2], node[3], 0.1)  # L2 miss rate

# Memory always hits (back to CPU)
P.set(jobclass, jobclass, node[3], node[0], 1.0)
model.link(P)

In [8]:
# Solve
solver = SolverMVA(model)
AvgTable = solver.getAvgNodeTable()

MVA [method: default(exact), lang: java, env: 21.0.7] completed in 0.000000s.
      Node  JobClass    QLen    Util   RespT  ResidT     ArvR     Tput
0      CPU  Requests  4.0514  4.0514  0.1000  0.1000  40.5139  40.5139
1  L1Cache  Requests  0.5774  0.4051  0.0143  0.0143  40.5139  40.5139
2  L2Cache  Requests  0.1856  0.1621  0.0229  0.0046   8.1028   8.1028
3   Memory  Requests  0.1856  0.1621  0.2291  0.0046   0.8103   0.8103
